# Ai Academy Capstone Group 4
Presented by Brian Gorbea, Kelvin Cupay, Robert Cofer, Rob Masters, Ryan Lazar

## Overview
Using Explanatory data analysis(EDA) and statistical methods we are going to advice Computing Vision how the best way to jump start their new movie studio, called 'Computing Vision Entertainment'. We used sqlite3 in pandas to query the datasets given and hypothesis testing to evaluate our findings. Essentially, we are generating insights for their business and stake holders by making a two part suggestion, one is the early stage of the movie studio and scalability after several years.



## Business Understanding: 

Goals: The business question we are going to answer
Assumption : The parent company 'Computing Vision' does not have much backgrounk in creating movies.
1. Best investment in order to quick start the revenue of new movie studio 'Computing Vidion Entertainment'
2. Best investment in order to scale the business and future ventures in the business industry

This notebook will drive deeper into how we utilize the data sets to create meaningful visualizations and aggregrations in order to achieve our goals listed above.

We will also create hypothesis test to evaluate our findings and reccomendations which are based in scietific & mathematical reasonings. The reccomendation for the *head of Computing Vision's new movie studio* and help him decide which films to create the best film in terms of profits and appreciation.

### Real World Application(real-world problem): Today less people are going to the theatres and watching movies. Maybe Hollywood has gone stale and have stagnanted with their ways of creating movies, maybe through insights of what is most profitable for a new and small scale movie studio someone can use this information to create and start their new movie business.REVISIT)

## Stakeholders:(REVISIT CHECK WITH ROB)
- `Investors`: Can identify by profits if rate on investment(ROI) is significant for them to invest in Computing Vision Entertainment
- `Computing Vision(Parent Company)`: Is it worth it to make 
- `Critics`: Is this new movie studio just for profits or they actually want to create movies that are well liked
- `Fans`: Do we like the movies from this mobie studio
- `Potential future fans`: Repuation of this new in coming movie studio
- `Competing fans`: How well will this Computing Vision Entertainment handle the competing movie studios

## Data Undertanding: 
The data set comes from several sources where some are compressed into CSV(comma-separated values) or TSV(tab-separated values). The data sets listed below are from these respectable sources: [Box Office Mojo](https://www.boxofficemojo.com/), [IMDB](https://www.imdb.com/), [Rotten Tomatoes](https://www.rottentomatoes.com/), [The MovieDB](https://www.themoviedb.org/), [The Numbers](https://www.the-numbers.com/).

Here is a list of the data sets use:
- `bom.movie_gross.csv.gz`: each record is a movie with a title(object), domestic_gross(object), foreign_gross(float), and year released(int)
- `im.db.zip`: 
- `rt.movie_info.tsv.gz`:
- `rt.reviews.tsv.gz`:
- `tmdb.movies.csv.gz`:
- `tn.movie_budgets.csv.gz:`


# this should be in data prep?
Task: exploring what types of films are current doing the best at the box office using different samples of available data(we are looking at profitability) : $defined$ $as$ global gross revenue() - budget(cost).
$create profit$
 gross_budgets['profit'] = gross_budgets['worldwide_gross'](revenue made international) - gross_budgets['production_budget'](cost)

# Middle : Methodology
* Data Understanding
    
* Data Analysis
    * 3 visualizations
* Statistical Inference

* Rationale
    * For example, why are you using hypothesis testing rather than just a graph?
    * What about the problem or data is suitable for this form of analysis?
    * For a data science audience, this includes your reasoning for the changes you applied and choices you made while building confidence intervals and/or hypothesis tests.
 

# End
* Recommendations
    * 3 reccs
* Next Steps
* Thank You
* This slide should include a prompt for questions as well as your contact information (name and LinkedIn profile)

# End of Note book

# Grading
* Attention to Detail (20%)
* Data Communication (20%)
* Authoring Jupyter Notebooks (20%)
* Data Manipulation and Analysis with pandas (20%)
* Statistical Communication (20%)

# This is for presentation
## Who will be receiving your presentation, and what is their role in the company?
* stake holders and investors of new movie Film Studio(top quality)

## What is their background: technical or non-technical?
non-technical, but they are sharks of Hollywood

## How will your presentation affect their work or a decision they need to make?
* Weather they will invest in the new company

## What is their attention span?
They have good attention span and good atttention to detail

## What is the goal of the presentation?
The goal of this presentation is for them to invest into Top Quality Entertainment


# from ROB

I think the takeways so far are:

- Action and kids movies are the top profit sources for major movies
- horror is the best route for under 10 million for a production budget
- It is important to pick the right studio as in the hypothesis testing each of the top 3 occurring were significant with profit
- Sci-Fi, adventure animation are the top profiting genres (assuming correct with data limitations)
- Under 10 mil budgets can expect a profit between 15-26 million (95% confidence)
- Over 10 mil budgets can expect profits between 116-143 million (95% confidence)